# Deep NLP - Word Embeddings

Think back to NLP as we've understood it so far.

If we've had some luck with NLP modeling, likely with a NaiveBayes algorithm, we were able to illustrate some correlations between words and some other feature of interest.

But to whatever extent that our models were able to make connections and pick up on correlations, they did this *without any understanding of the **meaning** of the words in question*.

Let's think for a minute about words and objective meanings!

We can make sense of meaning for computational purposes by thinking about meaning in terms of similarity, i.e. thinking about meaning *holistically*.

Q. Is there any precedent for this way of thinking about meaning? <br/>
A. [Yes](https://plato.stanford.edu/entries/meaning-holism/#ArgForMeaHol)

So what will this look like for us?

*Remember cosine similarity?*

$\rightarrow$We'll have much the same idea here: Associate each word with values along particular dimensions in a multi-dimensional space. If we had a dimension for *softness*, for example, then pillows and marshmallows would score higher on it than rocks and bricks.

In [1]:
import gensim
import numpy as np

What is Gensim? See [here](https://en.wikipedia.org/wiki/Gensim) and [here](https://radimrehurek.com/gensim/). But, basically, gensim is a package with lots of topic-modeling and NLP tools, inlcuding Word2Vec.

Find the data [here!](https://drive.google.com/file/d/0BwT5wj_P7BKXb2hfM3d2RHU1ckE/view) (Just click 'Download')

In [2]:
# Reading in the data

import json

with open('JEOPARDY_QUESTIONS1.json') as f:
    data = json.load(f)

In [3]:
type(data)

list

In [4]:
len(data)

216930

In [5]:
# Let's look at the first element in our list

data[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [6]:
# How many words do we have?

len(data[0]['question'])

98

In [7]:
# Let's try that again!

data[0]['question'].split(' ')

["'For",
 'the',
 'last',
 '8',
 'years',
 'of',
 'his',
 'life,',
 'Galileo',
 'was',
 'under',
 'house',
 'arrest',
 'for',
 'espousing',
 'this',
 "man's",
 "theory'"]

In [8]:
len(data[0]['question'].split(' '))

18

In [9]:
length = 0
for clue in data:
    length += len(clue['question'].split(' '))
length

3169994

## Using Word2Vec

In [12]:
# Word2Vec requires that our text have the form of a list
# of 'sentences', where each sentence is itself a list of
# words. How can we put our _Jeopardy!_ clues in that shape?

import string
text = []

for clue in data:
    sentence = clue['question'].translate(str.maketrans('', '',
                                                        string.punctuation)).split(' ')
    
    new_sent = [word.lower() for word in sentence]
    set_sent = set(new_sent)
    
    text.append(list(set_sent))

In [13]:
# Let's check the new structure of our first clue

text[0]

['last',
 'for',
 'of',
 'theory',
 'this',
 'was',
 'his',
 'years',
 'galileo',
 'under',
 '8',
 'the',
 'house',
 'arrest',
 'espousing',
 'mans',
 'life']

In [14]:
# Constructing the model is simply a matter of
# instantiating a Word2Vec object.

model = gensim.models.Word2Vec(text, sg=1)

Continuous Bag of Words vs. Skipgram

<a href="https://www.researchgate.net/figure/Illustration-of-the-Skip-gram-and-Continuous-Bag-of-Word-CBOW-models_fig1_281812760"><img src="https://www.researchgate.net/profile/Wang_Ling/publication/281812760/figure/fig1/AS:613966665486361@1523392468791/Illustration-of-the-Skip-gram-and-Continuous-Bag-of-Word-CBOW-models.png" alt="Illustration of the Skip-gram and Continuous Bag-of-Word (CBOW) models."/></a>

[More on Skipgram](https://towardsdatascience.com/word2vec-skip-gram-model-part-1-intuition-78614e4d6e0b)

In [15]:
# To train, call 'train()'!

model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(11006798, 14853590)

In [16]:
# Checking word count

model.corpus_total_words

2970718

## model.wv

In [17]:
# The '.wv' attribute stores the word vectors

model.wv

In [18]:
# The vectors are keyed by the words

model.wv['child']

array([-0.18835004, -0.10051052,  0.56742996, -0.38782132,  0.05395151,
        0.30893523,  0.01885724,  0.00512805,  0.34733975, -0.42252317,
       -0.02703452, -0.37947437, -0.02648863, -0.22058631, -0.12377359,
        0.3034612 ,  0.19703291,  0.43381453, -0.01943327,  0.1378722 ,
       -0.12958533,  0.19817032,  0.40009233, -0.18560956, -0.03262196,
       -0.03280225,  0.7176073 , -0.68748313,  0.2696547 ,  0.236288  ,
        0.05716058, -0.15227845,  0.02512128,  0.09427069,  0.12817399,
       -0.14056052,  0.28812647, -0.07106934, -0.07075839,  0.32485816,
       -0.26228067,  0.57501024,  0.20029369,  0.3513924 ,  0.33409947,
        0.33749396,  0.18212356, -0.19127296,  0.97023445, -0.19185641,
       -0.01827691, -0.13421504,  0.33293837, -0.09672356,  0.61728305,
       -0.1630977 ,  0.24887839,  0.19782183, -0.09601402, -0.47092262,
       -0.14621574, -0.04865019,  0.06088542, -0.12594233,  0.30159527,
       -0.24800242,  0.28298506, -0.08135349, -0.152371  , -0.16

In [19]:
model.wv['child'].shape

(100,)

Size is 100 because the default is 100. Usually reasonable, but should go up as the number of words goes up. 

### model.wv methods
#### 'most_similar()' and 'similarity()'

In [20]:
model.wv.most_similar('furniture')

[('linen', 0.7968616485595703),
 ('ornaments', 0.7818689346313477),
 ('dutchborn', 0.7670450210571289),
 ('nativity', 0.7597681283950806),
 ('decorative', 0.7596691846847534),
 ('millet', 0.758497953414917),
 ('crafted', 0.7544798851013184),
 ('supplement', 0.7515498399734497),
 ('unglazed', 0.7466052770614624),
 ('integral', 0.7461913824081421)]

In [21]:
model.wv.similarity('furniture', 'jewelry')

0.5330559

In [22]:
model.wv.most_similar(positive=['cat', 'animal', 'pet', 'mammal'])

[('carnivore', 0.8060418367385864),
 ('endangered', 0.800355851650238),
 ('preys', 0.786952555179596),
 ('feline', 0.7785636186599731),
 ('marsupial', 0.7752657532691956),
 ('ivorybilled', 0.7725008130073547),
 ('scavenger', 0.7683324813842773),
 ('vulpes', 0.7597392201423645),
 ('beetle', 0.7591995000839233),
 ('horned', 0.7591184377670288)]

Consider the following 'equations':

King + Woman - Man = x

Brother + Woman - Man = y

What values would you suggest for x and y here?

Clearly, getting good answers to these equations depends on understanding the *meanings* of the underlying words.

Or does it? The `most_similar()` method takes a 'negative' parameter as well as a 'positive' one, so we can consult our trained word vectors to see how they would answer these questions!

In [23]:
model.wv.most_similar(positive=['cat', 'animal'], negative='pet')

[('domesticated', 0.3434196710586548),
 ('breed', 0.3179727792739868),
 ('bird', 0.2775746285915375),
 ('grow', 0.2703922390937805),
 ('ago', 0.2653577923774719),
 ('species', 0.26091259717941284),
 ('arachnid', 0.2581769526004791),
 ('creatures', 0.2569670081138611),
 ('dog', 0.25542062520980835),
 ('extinct', 0.2506164312362671)]

In [25]:
model.wv.most_similar(positive=['king', 'woman'], negative='man')#, topn=3)

[('throne', 0.28611981868743896),
 ('bride', 0.2642226815223694),
 ('empress', 0.26002126932144165),
 ('heir', 0.23214277625083923),
 ('heroine', 0.22555987536907196),
 ('princess', 0.2192891389131546),
 ('reigned', 0.2105431854724884),
 ('herself', 0.20507055521011353),
 ('husband', 0.18874485790729523),
 ('constantine', 0.18830817937850952)]

In [26]:
model.wv.most_similar(positive='usa')

[('fargo', 0.6736178398132324),
 ('coanchor', 0.6470738649368286),
 ('registry', 0.6416285037994385),
 ('viacom', 0.6360081434249878),
 ('tyson', 0.635170042514801),
 ('bruin', 0.6328413486480713),
 ('cnbc', 0.6272039413452148),
 ('sweetheart', 0.625778317451477),
 ('drivein', 0.6234688758850098),
 ('jacksonville', 0.6191979050636292)]

In [27]:
model.wv.most_similar('canada')

[('eh', 0.677048921585083),
 ('palestinian', 0.6678165197372437),
 ('indonesia', 0.6666308641433716),
 ('northwestern', 0.664191722869873),
 ('venezuela', 0.6622143983840942),
 ('nez', 0.6605390310287476),
 ('zimbabwe', 0.6599727272987366),
 ('indianas', 0.6599428653717041),
 ('timor', 0.6578361988067627),
 ('ceuta', 0.6559556722640991)]

In [ ]:
model.wv.most_similar('shakespeare')

In [ ]:
model.wv.most_similar('greg')

In [ ]:
model.wv.most_similar('jefferson')

In [ ]:
model.wv.most_similar('washington')

In [28]:
model.wv.most_similar(positive=['president', 'germany'], negative='usa')

[('exile', 0.2267046421766281),
 ('emperors', 0.22108100354671478),
 ('emperor', 0.220118448138237),
 ('romes', 0.2022033929824829),
 ('byzantine', 0.2013220489025116),
 ('assassinated', 0.18931180238723755),
 ('tokyo', 0.18898290395736694),
 ('construction', 0.18593069911003113),
 ('conquest', 0.18502110242843628),
 ('milan', 0.18497797846794128)]

In [29]:
model.wv.most_similar(positive=['president', 'france'], negative='usa')

[('exile', 0.20584151148796082),
 ('conquest', 0.19162502884864807),
 ('assassinated', 0.18330737948417664),
 ('pope', 0.16349422931671143),
 ('jerusalem', 0.16297361254692078),
 ('deposed', 0.16267861425876617),
 ('emperors', 0.16242769360542297),
 ('minister', 0.15971383452415466),
 ('mecca', 0.15835599601268768),
 ('romes', 0.15732252597808838)]

#### 'doesnt_match()'

In [30]:
model.wv.doesnt_match(['breakfast', 'lunch', 'frog', 'food'])

//anaconda3/envs/learn-env/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'frog'

In [31]:
model.wv.doesnt_match(['lunch', 'this'])

//anaconda3/envs/learn-env/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'this'

In [32]:
model.wv.doesnt_match(['tree', 'flower', 'bush', 'plant', 'toothbrush'])

//anaconda3/envs/learn-env/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'bush'

In [33]:
model.wv.doesnt_match(['tree', 'flower', 'plant', 'toothbrush'])

//anaconda3/envs/learn-env/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'toothbrush'

#### 'closer_than()'

In [34]:
# Which words are closer to 'king' than 'queen' is?

model.wv.closer_than('king', 'queen')

['prince',
 'emperor',
 'kings',
 'pope',
 'iv',
 'vi',
 'viii',
 'crowned',
 'frederick',
 'vii',
 'gustav',
 'stuart',
 'dane',
 'tudor',
 'isabella',
 'quest',
 'coronation',
 'reigning',
 'adrian',
 'cyrus',
 'viiis',
 'xiii',
 'iis',
 'czars',
 'alfonso',
 '1547',
 'handel',
 '1742',
 'umberto',
 '1760',
 'prophecy',
 'olaf',
 'hittite',
 'centurys',
 '1540',
 '1653',
 'trickster',
 'lionhearted',
 'courtier',
 'inquisitor',
 'pepin',
 '1589',
 'moliere',
 'saxon',
 'deacon',
 '1631',
 'ides',
 'supremacy',
 '1632',
 'wilhelmina',
 'anointed',
 'rightful',
 'martel',
 'singh',
 'martyrdom',
 '1509',
 '1599',
 'henrik',
 'henrietta',
 'sforza',
 'affirmed',
 '331',
 'olav',
 'plutarch',
 'canute',
 'evacuated',
 '1647',
 'cranmer',
 '751',
 'sihanouk',
 'theban',
 'cato',
 'wisest',
 '1523',
 'heretic',
 'agrippa',
 'camillas',
 'stirling']

#### 'distance()'

In [35]:
# For this it will make more sense to
# normalize our vectors.

model.init_sims(replace=True)

In [36]:
model.wv.distance('king', 'king')

0.0

In [37]:
model.wv.distance('joy', 'happiness')

0.6370022892951965

#### 'evaluate_word_analogies()'

The `evaluate_word_analogies()` method takes in a string of quadruples, properly formatted (see [here](https://radimrehurek.com/gensim/models/keyedvectors.html)), and returns a list of dictionaries. Each dictionary has two keys: 'correct' and 'incorrect', the values for which are lists of the analogies that the model correctly or incorrectly predicted.

Check out [this text file](https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt)!

In [40]:
gensim.__version__

'3.8.0'

In [39]:
relatives = model.wv.evaluate_word_analogies("""https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt""")[1][4]

In [41]:
len(relatives['correct'])

41

In [42]:
len(relatives['incorrect'])

379

In [43]:
relatives['correct'][:5]

[('BOY', 'GIRL', 'FATHER', 'MOTHER'),
 ('BOY', 'GIRL', 'HIS', 'HER'),
 ('BOY', 'GIRL', 'KING', 'QUEEN'),
 ('BOY', 'GIRL', 'SON', 'DAUGHTER'),
 ('BOY', 'GIRL', 'UNCLE', 'AUNT')]

In [44]:
relatives['incorrect'][:5]

[('BOY', 'GIRL', 'BROTHER', 'SISTER'),
 ('BOY', 'GIRL', 'BROTHERS', 'SISTERS'),
 ('BOY', 'GIRL', 'DAD', 'MOM'),
 ('BOY', 'GIRL', 'GRANDFATHER', 'GRANDMOTHER'),
 ('BOY', 'GIRL', 'GRANDPA', 'GRANDMA')]